In [ ]:
!cat *.csv > data.csv

In [ ]:
!cat /content/CSV/1.Deauth/*.csv > 1.csv
!cat /content/CSV/2.Disas/*.csv > 2.csv
!cat /content/CSV/3.(Re)Assoc/*.csv > 3.csv
!cat /content/CSV/4.Rogue_AP/*.csv > 4.csv
!cat /content/CSV/5.Krack/*.csv > 5.csv
!cat /content/CSV/6.Kr00k/*.csv > 6.csv
!cat /content/CSV/7.SSH/*.csv > 7.csv
!cat /content/CSV/8.Botnet/*.csv > 8.csv
!cat /content/CSV/9.Malware/*.csv > 9.csv
!cat /content/CSV/10.SQL_Injection/*.csv > 10.csv
!cat /content/CSV/11.SSDP/*.csv > 11.csv
!cat /content/CSV/12.Evil_Twin/*.csv > 12.csv

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `cat /content/CSV/3.(Re)Assoc/*.csv > 3.csv'


In [ ]:
ls

sample_data/


In [ ]:
!wget https://icsdweb.aegean.gr/awid/download.php?token=195ca45fd9c1f206318db7d755dd74f64a0997e6554bc220bbefae730b2f7cd5 -O DATASET.tar.gz

--2023-04-27 20:17:51--  https://icsdweb.aegean.gr/awid/download.php?token=195ca45fd9c1f206318db7d755dd74f64a0997e6554bc220bbefae730b2f7cd5
Resolving icsdweb.aegean.gr (icsdweb.aegean.gr)... 195.251.134.102
Connecting to icsdweb.aegean.gr (icsdweb.aegean.gr)|195.251.134.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14471343012 (13G) [application/octet-stream]
Saving to: ‘DATASET.tar.gz’

DATASET.tar.gz      100%[===================>]  13.48G  9.55MB/s    in 25m 8s  

2023-04-27 20:43:00 (9.15 MB/s) - ‘DATASET.tar.gz’ saved [14471343012/14471343012]



In [ ]:
!unzip /content/DATASET.zip

In [ ]:
import numpy as np
import pandas as pd

def load_data(file_path):
    '''Fuction to load csv data file as dataframe from given file path'''
    df = pd.read_csv(file_path)
    return df

df = load_data('/content/data.csv')
print('Shape of data: ', df.shape)
df.head()

In [ ]:
df.describe()


NameError: ignored

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

def class_distrib(df):
    '''This function displays class distribution as a bar plot for the given dataframe'''
    unique, counts = np.unique(df['class'], return_counts=True)
    distrib = dict(zip(unique, counts))

    plt.figure(figsize=(10, 8))
    sns.barplot(x=list(distrib.keys()), y=list(distrib.values()))
    plt.title("Class Distribution")
    plt.xticks(rotation=45)
    plt.xlabel("Classes")
    plt.ylabel("Count")

    plt.show()
    print(distrib)
    
class_distrib(df)

In [ ]:

def subset_sample(df, class_name, n):
    '''This function extracts n range subset from the given dataframe for the given input class'''
    df_maj = df[df['class'] == class_name]
    df = df[df['class'] != class_name]

    df_maj = df_maj[100 : n]

    df = pd.concat([df_maj, df], axis=0)

    return df

df = subset_sample(df, 'spoof', 200000)
class_distrib(df)

In [ ]:
# Rows which still have some NaN value 
df_nan_rows = df[df.isnull().any(axis=1)]
print('All class rows which still have some NaN value: ', len(df_nan_rows))

df_nan_rows_normal = df[df.isnull().any(axis=1)][df['class'] == 'normal']
print('Normal class rows which still have some NaN value: ', len(df_nan_rows_normal))

In [ ]:
def shallow_clean(df):
    '''Cleans the the given dataframe at a shallow level'''
    # Remove columns with >= 85% NaNs
    thres_limit = len(df) * 0.25
    df = df.dropna(thresh=int(thres_limit), axis=1)

    # Remove columns with constant value
    df = df.drop(columns=df.columns[df.nunique()==1])

    # Remove rows with >= 50% NaNs
    thres_limit = df.shape[1] * 0.50
    df = df.dropna(thresh=int(thres_limit), axis=0)
    
    return df

cols_before = set(df.columns)
df = shallow_clean(df)
cols_after = set(df.columns)
print('Columns removed:', cols_before - cols_after)

In [ ]:
df.isna().any()

In [ ]:
df.dtypes


In [ ]:
class_distrib(df_nan_rows)


In [ ]:
#Fill the NULL values
num_nan_cols = ['radiotap.mactime', 'radiotap.datarate', 'wlan.frag', 'wlan.seq']
for c in num_nan_cols:
    df[c].fillna(value=df[c].mean(), inplace=True)
    
cat_nan_cols = ['wlan.ra', 'wlan.da', 'wlan.ta', 'wlan.bssid', 'wlan.sa']
for c in cat_nan_cols:
    df[c].fillna(value='null', inplace=True)
    
df.columns[df.isna().any()].tolist()

In [ ]:
cols_drop = ['wlan.da', 'wlan.ra', 'wlan.ta', 'wlan.sa', 'wlan.bssid', 'radiotap.dbm_antsignal']
df = df.drop(cols_drop, axis=1)

In [ ]:
# Save processed data to file
df.to_csv('data/train_processed_1.csv', index=False)

In [ ]:

# Load processed data to df
df = pd.read_csv('data/train_processed_1.csv')

In [ ]:

X = df.drop(['class'], axis=1)
y = df['class']

In [ ]:
from sklearn.preprocessing import StandardScaler
import joblib

std_cols = ['frame.time_epoch', 'frame.time_delta', 'frame.time_delta_displayed', 'frame.time_relative', 'frame.len',
           'frame.cap_len', 'radiotap.length', 'radiotap.mactime', 'radiotap.datarate', 'radiotap.channel.freq',
           'wlan.duration', 'wlan.frag', 'wlan.seq']
with open('scaler_save/columns.txt', 'w') as f:
    [f.write(c+'\n') for c in std_cols]

standard_scaler = StandardScaler()
X[std_cols] = standard_scaler.fit_transform(X[std_cols])

# Save this scalar to file
joblib.dump(standard_scaler, 'scaler_save/scaler.gz')

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc_cols = ['wlan.fc.type_subtype', 'wlan.fc.type', 'wlan.fc.subtype', 'wlan.fc.ds', 'radiotap.present.tsft']

with open('encoder_save/columns.txt', 'w') as f:
    [f.write(c+'\n') for c in enc_cols]
    
oh_encoder_x = OneHotEncoder(sparse=False, handle_unknown='ignore')
array_ohe = oh_encoder_x.fit_transform(X[enc_cols])
df_ohe = pd.DataFrame(array_ohe, index=df.index)

#Extract only the columns that didnt need to be encoded
df_other = X.drop(columns=enc_cols)

#Concatenate the two dataframes : 
X = pd.concat([df_ohe, df_other], axis=1)

# Save this encoder to file
joblib.dump(oh_encoder_x, 'encoder_save/encoder.gz')

In [ ]:

print('Shape of y before encoding: ', y.shape)
print('ReShape of y for encoding: ', y.values.reshape(-1, 1).shape)

oh_encoder_y = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_categorical = oh_encoder_y.fit_transform(y.values.reshape(-1, 1))
print('Shape of y after encoding: ', y_categorical.shape)

In [ ]:

df.shape, X.shape, y.shape, y_categorical.shape

In [ ]:

def time_series(X, chunk_size):
    si = chunk_size
    ei = len(X)
    
    data = []

    for i in range(si, ei):
        idx = range(i - chunk_size, i, 1)
        data.append(X[idx])

    data = np.array(data)
    return data

chunk_size = 10
X_time = time_series(X.values, chunk_size)
X_time.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_time, y_categorical[chunk_size:], test_size=0.33, stratify=y[chunk_size:])
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.33, stratify=y_val)

In [ ]:

relevant_columns = df.columns
with open('data/relevant_columns.txt', 'w') as f:
    [f.write(rc+'\n') for rc in relevant_columns]

In [ ]:

class_labels = oh_encoder_y.categories_[0]
with open('data/class_labels.txt', 'w') as f:
    [f.write(cl+'\n') for cl in class_labels]

In [ ]:

X_train.shape[1:3]

In [ ]:

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

def create_model(input_shape, n_class):
    """This function creates a simple DL model"""
    input_layer = tf.keras.Input(shape=(input_shape))
    
    gru_layer = layers.GRU(16, return_sequences=False)(input_layer)
    
    dense_layer1 = layers.Dense(16, activation='relu')(gru_layer)
    dense_layer2 = layers.Dense(32, activation='relu')(dense_layer1)
    
    output_layer = layers.Dense(n_class, activation='softmax')(dense_layer2)
    return Model(input_layer, output_layer)
    
model = create_model(X_train.shape[1:3], len(oh_encoder_y.categories_[0]))

model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])


In [ ]:

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
%load_ext tensorboard

log_dir="tensorboard\\logs"
tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True)

checkpoint = ModelCheckpoint(filepath='model_save/best_model.h5', monitor='val_accuracy',
                             verbose=1, save_best_only=True, mode='auto')

earlystop = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=4, verbose=1)

In [ ]:
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val),
              batch_size=128, callbacks=[checkpoint, earlystop, tensorboard])

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

In [ ]:

from sklearn.metrics import confusion_matrix, classification_report

y_preds_val = np.argmax(model.predict(X_val), axis=1)
y_val = np.argmax(y_val, axis=1)
y_preds_test = np.argmax(model.predict(X_test), axis=1)
y_test = np.argmax(y_test, axis=1)
y_preds_train = np.argmax(model.predict(X_train), axis=1)
y_train = np.argmax(y_train, axis=1)

with open('data/class_labels.txt', 'r') as f:
    class_labels = np.array([line.strip() for line in f])

cm_val = confusion_matrix(class_labels[y_val], class_labels[y_preds_val])
cm_test = confusion_matrix(class_labels[y_test], class_labels[y_preds_test])
cm_train = confusion_matrix(class_labels[y_train], class_labels[y_preds_train])

df_cm_val = pd.DataFrame(cm_val, columns=class_labels, index=class_labels)
df_cm_train = pd.DataFrame(cm_train, columns=class_labels, index=class_labels)
df_cm_test = pd.DataFrame(cm_test, columns=class_labels, index=class_labels)
df_cm_val.index.name = df_cm_train.index.name = df_cm_test.index.name = 'Actual'
df_cm_val.columns.name = df_cm_train.columns.name = df_cm_test.columns.name = 'Predicted'

In [ ]:
fig = plt.figure(figsize = (14, 24))

fig.add_subplot(3, 1, 1)
sp2 = sns.heatmap(df_cm_train, cmap="Blues", annot=True, fmt='g')
sp2.set_title('Traning CM')

fig.add_subplot(3, 1, 2)
sp1 = sns.heatmap(df_cm_val, cmap="Blues", annot=True, fmt='g')
sp1.set_title('Validation CM')

fig.add_subplot(3, 1, 3)
sp2 = sns.heatmap(df_cm_test, cmap="Blues", annot=True, fmt='g')
sp2.set_title('Testing CM')

plt.show()

In [ ]:

print(classification_report(class_labels[y_val], class_labels[y_preds_val], target_names=list(class_labels)))
print(classification_report(class_labels[y_test], class_labels[y_preds_test], target_names=list(class_labels)))